In [16]:
import pandas as pd 
import numpy as np 
import torch

In [114]:
dataset_directory = "/Users/erikharaldson/Documents/FindAI/DataSets/"
image_directory = "/Users/erikharaldson/Documents/FindAI/Images/"

In [79]:
df_item_info = pd.read_csv(dataset_directory + "ProductStructuredInfo.csv")
descriptive_embeddings = pd.read_csv(dataset_directory + "DescriptiveEmbeddings.csv", header=None)
keyword_embeddings = pd.read_csv(dataset_directory + "KeywordEmbeddings.csv", header=None)
image_embeddings = pd.read_csv(dataset_directory + "ImageEmbeddings.csv", header=None)
structured_embeddings = pd.read_csv(dataset_directory + "StructuredEmbeddings.csv", header=None)

In [80]:
descriptive_embeddings_torch = torch.tensor(descriptive_embeddings.values, dtype=torch.float32)
image_embeddings_torch = torch.tensor(image_embeddings.values, dtype=torch.float32)
keyword_embeddings_torch = torch.tensor(keyword_embeddings.values, dtype=torch.float32)
structured_embeddings_torch = torch.tensor(structured_embeddings.values, dtype=torch.float32)

In [138]:
import torch
import clip

# Test for text retrieval
device = "cpu"

# Load the pre-trained CLIP model and its tokenizer.
model, preprocess = clip.load("ViT-B/32", device=device)

text = "navy"

text_tokens = clip.tokenize(text).to(device)
text_embeddings = model.encode_text(text_tokens)
text_embeddings = text_embeddings.mean(dim=0, keepdim=True)
text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)
text_embeddings = text_embeddings.reshape(-1)

In [130]:
from PIL import Image

# Test for image retrieval
image = Image.open(image_directory + "coatjpeg.jpg")
image_input = preprocess(image).unsqueeze(0).to(device)

with torch.no_grad():
    image_features = model.encode_image(image_input)

image_features /= image_features.norm(dim=-1, keepdim=True)

img_embedding = image_features.reshape(-1)

In [135]:
def average_embeddings(embedding1, embedding2):
    
    # Check if embeddings have the same shape
    if embedding1.shape != embedding2.shape:
        raise ValueError(f"Embedding shapes don't match: {embedding1.shape} vs {embedding2.shape}")
    
    # Calculate the average embedding
    average_embedding = (embedding1 + embedding2) / 2
    
    return average_embedding

In [139]:
# Mix text and image embeddings
mixed_embeddings = average_embeddings(text_embeddings, img_embedding)

In [140]:
def retrieve_image_urls_top_k(product_info: pd.DataFrame, input_embedding, embedding_database, top_k = 3):

    cosine_similarity_matrix = input_embedding @ embedding_database.T
    cosine_similarity_matrix = torch.nan_to_num(cosine_similarity_matrix, nan=0.0)

    similar_indices = torch.topk(cosine_similarity_matrix, top_k).indices.cpu().numpy()

    image_urls = []

    for indx in similar_indices:

        first_url = product_info.iloc[indx]["image_urls"].split("'")[1]
        image_urls.append(first_url)

    return image_urls

In [141]:
retrieve_image_urls_top_k(product_info=df_item_info, input_embedding=mixed_embeddings, embedding_database=image_embeddings_torch)

['https://media.endclothing.com/media/f_auto,q_auto:eco,w_200/prodmedia/media/catalog/product/2/3/23-09-24-LS_WN-J002-051-1_1_1.jpg',
 'https://media.endclothing.com/media/f_auto,q_auto:eco,w_200/prodmedia/media/catalog/product/0/8/08-08-2023-JA_232-020206-7228-900_1_1.jpg',
 'https://media.endclothing.com/media/f_auto,q_auto:eco,w_200/prodmedia/media/catalog/product/2/4/24-06-24-ns2_242-020219-9223-900_1.jpg']